In [1]:
import pandas as pd

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
torch.cuda.is_available()

True

In [3]:
LR = 2e-5
EPOCHS = 1
BATCH_SIZE = 16
MODEL = "cardiffnlp/twitter-xlm-roberta-base"
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

In [4]:
train = pd.read_csv('./Multiple_sources/processed_train.csv').dropna()
x_train = train['text']
y_train, _ = train['label'].factorize()
test = pd.read_csv('./Multiple_sources/processed_test.csv').dropna()
x_test = test['text']
y_test, _ = test['label'].factorize()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1, stratify=y_train)

In [5]:
x_train

374539    içindeki zil kedimin epey topun pesinden gidip...
383411    ürün bugün elime ulaştı french bulldog cinsi k...
215734                      beğendim ambalajlı geldi tşkler
432730    taş kağıt makasa forma kaybeden maldır amk hay...
316221    Köyün yerleşime açık Bahçecik adında bir mezra...
                                ...                        
141501    Sezonun başında Şampiyonlar Ligi eleme maçları...
226933    Ruhsal portreler kişilerin görülebilen özellik...
232308             ürün güzel kurulumu kolay tavsiye ederim
284265    Eski yapıların bulunduğu alan Kent Meydanı ola...
399420                            kesinlikle tavsiye ederim
Name: text, Length: 396576, dtype: object

In [67]:
y_train[y_train == 0] = 3
y_train[y_train == 1] = 0
y_train[y_train == 3] = 1
y_train

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [6]:
x_test

0              Kral akbaba dikkat çekici renklere sahiptir
1        ısrarla korkutmayı başarıyor korku uzun bitmiy...
2        Neşe Üzüntü köprünün kırılmaya başlamasıyla ge...
3        i phone 5 ten sonra gene 4 ekranı tercih ettim...
4            Beşinci sezonda diziye yeni oyuncular katıldı
                               ...                        
48960    Fransa bayrağı diğer kırmızı zeminden beyaz bi...
48961    Yine aynı yıl türkü dalında Murat Çobanoğlu bi...
48962                             Kurgunu skiyim oç evladı
48963    Şarkı sonrasında Damian Marley tarafından sesl...
48964    berrak bir ürün ancak kendi orijinal spigen ly...
Name: text, Length: 48960, dtype: object

In [27]:
y_test

array([0, 1, 0, ..., 2, 0, 2], dtype=int64)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

In [9]:
x_train = x_train.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=128))
x_val = x_val.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=128)) 
x_test = x_test.apply(lambda x: tokenizer(x, truncation=True, padding="max_length", max_length=128))

In [10]:
x_train.reset_index(drop=True, inplace=True)

In [11]:
x_val.reset_index(drop=True, inplace=True)
x_val

0        [input_ids, attention_mask]
1        [input_ids, attention_mask]
2        [input_ids, attention_mask]
3        [input_ids, attention_mask]
4        [input_ids, attention_mask]
                    ...             
44059    [input_ids, attention_mask]
44060    [input_ids, attention_mask]
44061    [input_ids, attention_mask]
44062    [input_ids, attention_mask]
44063    [input_ids, attention_mask]
Name: text, Length: 44064, dtype: object

In [12]:
x_test.reset_index(drop=True, inplace=True)

max_tok = 0
for i in x_train:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
3549

max_tok = 0
for i in x_val:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
2241

max_tok = 0
for i in x_test:
    if len(i['input_ids']) > max_tok:
        max_tok = len(i['input_ids'])
max_tok
1331

In [70]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {"input_ids": torch.tensor(self.encodings[idx]["input_ids"])}
        item['attention_mask'] = torch.tensor(self.encodings[idx]["attention_mask"])
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(x_train, y_train)
val_dataset = MyDataset(x_val, y_val)
test_dataset = MyDataset(x_test, y_test)

In [9]:
train_encodings = tokenizer(x_train.to_list(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(x_val.to_list(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(x_test.to_list(), truncation=True, padding=True, max_length=128)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, y_train)
val_dataset = MyDataset(val_encodings, y_val)
test_dataset = MyDataset(test_encodings, y_test)

In [14]:
training_args = TrainingArguments(
    output_dir='./results',                   # output directory
    num_train_epochs=EPOCHS,                  # total number of training epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # batch size for evaluation
    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,                         # when to print log
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

Some weights of the model checkpoint at cardiffnlp/twitter-xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['classifier.out

In [15]:
trainer = Trainer(
    model=model,                              # the instantiated 🤗 Transformers model to be trained
    args=training_args,                       # training arguments, defined above
    train_dataset=train_dataset,              # training dataset
    eval_dataset=val_dataset                  # evaluation dataset
)

trainer.train()

C:\Users\assas\anaconda3\envs\ml\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 396576
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 24786


Step,Training Loss
10,1.136500
20,1.073100
30,0.887300
40,0.742900
50,0.650400
60,0.470300
70,0.387500
80,0.304000
90,0.342100
100,0.324400


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-1500
Configuration saved in ./results\checkpoint-1500\config.json
Model weights saved in ./results\checkpoint-1500\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-2000
Configuration saved in ./results\checkpoint-2000\config.json
Model weights saved in ./results\checkpoint-2000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-2500
Configuration saved in ./results\checkpoint-2500\config.json
Model weights saved in ./results\checkpoint-2500\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-3000
Configuration saved in ./results\checkpoint-3

Saving model checkpoint to ./results\checkpoint-23000
Configuration saved in ./results\checkpoint-23000\config.json
Model weights saved in ./results\checkpoint-23000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-23500
Configuration saved in ./results\checkpoint-23500\config.json
Model weights saved in ./results\checkpoint-23500\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-24000
Configuration saved in ./results\checkpoint-24000\config.json
Model weights saved in ./results\checkpoint-24000\pytorch_model.bin
Saving model checkpoint to ./results\checkpoint-24500
Configuration saved in ./results\checkpoint-24500\config.json
Model weights saved in ./results\checkpoint-24500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=24786, training_loss=0.17074270684109427, metrics={'train_runtime': 6947.7463, 'train_samples_per_second': 57.08, 'train_steps_per_second': 3.567, 'total_flos': 2.608611668722483e+16, 'train_loss': 0.17074270684109427, 'epoch': 1.0})

In [42]:
#Val set has not been used yet. Test set gives similar result.(Add Test result too when running again.) 
val_preds_raw, val_labels , _ = trainer.predict(val_dataset)
val_preds = np.argmax(val_preds_raw, axis=-1)
print(classification_report(val_labels, val_preds, digits=3))

***** Running Prediction *****
  Num examples = 44064
  Batch size = 16


              precision    recall  f1-score   support

           0      0.959     0.972     0.965     23592
           1      0.990     0.998     0.994     15382
           2      0.883     0.809     0.844      5090

    accuracy                          0.962     44064
   macro avg      0.944     0.926     0.934     44064
weighted avg      0.961     0.962     0.961     44064



In [17]:
trainer.save_model("./results/best_model") # save best model

Saving model checkpoint to ./results/best_model
Configuration saved in ./results/best_model\config.json
Model weights saved in ./results/best_model\pytorch_model.bin


In [54]:
online_test = pd.Series(['Kurtuluş Savaşı İzmir in isgaliyle başlar, İzmirin kurtuluşu ile biter.',
                        'Seçimi kaybedeceğini bilen bir hükümet, "30 gün prim ödeyen herkes, yaşına bakılmaksızın ve derhal kişi başına düşen gelir kadar emekli aylığı almaya hak kazanır" diye bir kanun çıkarsa, bu kanun bir kere çıktı, kazanılmış hak oldu, herkese bu ödeme yapılsın diyecek misin?',
                        'Karınca kolonisi optimizasyonu her zaman iyi sonuç vermeyebilir.',
                        'Siyasilere manifesto: Benim paramla ulufe dağıtma yarışına giresiniz diye vergi ödemiyorum.',
                        'Ben ekonomideki cehaletin boyutlarını görünce artık yapıcı yıkımdan başka bir yol olmadığını anlıyorum. Maliyeti engellemek için tüm samimiyetimizle doğruyu söyledik ama artık popülizm tavan yapmış durumda. Hepimiz ödeyeceğiz bu maliyeti!',
                        'Dünyanın en büyük üçüncü ekonomisini en uzun süre yönetmiş adam suikaste uğruyor ancak bu olay medyada çok az yer alıyor. Maktül bu kadar büyükken bu kadar cılız ses çıkması şaşırtıcı.',
                        'Seni it oğlu köpek şerefsiz.',
                        'Bunu tasvip etmiyorum.',
                        'Çok kötü bir gelişme.',
                        'Bu hiçte kötü değil.',
                        'Rezalet.',
                        'Hisse artışa geçti çabuk alın.',
                        'Olley be bist100 uçtu.',
                        'Bist100 tavsiye ederim kaçırmayın.'])
online_test = online_test.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=128))
online_ds = MyDataset(online_test, [1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0])
trainer.predict(online_ds)

***** Running Prediction *****
  Num examples = 14
  Batch size = 16


PredictionOutput(predictions=array([[-2.7639523 ,  6.209841  , -3.5093257 ],
       [ 3.3379538 , -4.2822948 , -0.07614036],
       [-2.6974714 ,  6.151871  , -3.5292373 ],
       [ 1.5008385 , -2.9210513 ,  0.7395926 ],
       [ 2.81848   , -4.006406  ,  0.19193465],
       [-0.7852182 ,  3.0212512 , -2.063731  ],
       [-0.55728257, -3.1341426 ,  4.6073575 ],
       [ 0.01268318,  0.98467207, -0.9847228 ],
       [-0.84086907, -1.6498721 ,  3.7046628 ],
       [-1.0766885 ,  3.684284  , -2.416834  ],
       [-0.43949133, -3.2303922 ,  4.5779424 ],
       [-0.7951734 ,  2.7179723 , -1.6944032 ],
       [-0.79751134,  3.0040443 , -2.0193284 ],
       [ 4.650012  , -3.9620986 , -1.5205253 ]], dtype=float32), label_ids=array([1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0], dtype=int64), metrics={'test_loss': 1.9274097681045532, 'test_runtime': 0.2101, 'test_samples_per_second': 66.649, 'test_steps_per_second': 4.761})

In [56]:
online_test = pd.Series(['Arkadaşlar 1 yıldır değişmeyen takip listemin başarı oranım/ 100 durBöyle olunca aynısını değerli 😀😀😀trader...kopyala yapıştır misali takip listeleri olmuş  birebir😀😀 Ayıptır emektir ustanın biraz edep yahu edep.',
                        'İki aya kalmaz yüzde %400 gidecek iki #hisse gelecek #bist100 500 rt gelecek mi #bist30',
                        '#Bist100 #xu100 dolar bazlı; yeşil çizginin üstünde kaldı, yönünü yukarı çevirdi. Pembe çizgiyi boşlukla aşacağı, son yılların en güçlü rallisinin eşiğinde olabilir. Kısa vadeli hedefi 273. Orta vadeli hedefi 324. Yatırım tavsiyesi değildir. #Elliott',
                        'Bu hafta, yatırım araçlarından #borsa yatırımcıya kaybettirirken, #altın ve #dolar kazandırdı.Hafta içinde en düşük 3.048,67 puanı, en yüksek 3.319,14 puanı gören #bist100 endeksi haftayı %3,10 kayıpla 3.179,99 puandan tamamladı.',
                        '#BORSA Çenem düştü azıcık konuştum ister izleyin ister izlemeyin Sonra gidip uçan balonlara binin',
                        'Bugün akşam size pazartesinin tavanını yazacağım.Tavan hissem için bol BEĞENİ ve RT ile destek verin.Yeni 🔒 KİLİT hissemi bu akşam paylaşacağım🚀🚀🚀🚀🔒🔒🔒🔒#borsa #bist100 #tavan #rally',
                        'Yükseliş trendine geçerek ilk direnç 30.62 seviyelerini deneyip tekrar düşüş yaşayan #EREGL yatay olarak seyir göstermektedir.  Tekrar düşüş yaşaması durumunda 29.96 ve 29.66 destek seviyelerini test etmesi muhtemeldir.'])
online_test = online_test.apply(lambda x: tokenizer(x, truncation=True, padding='max_length', max_length=128))
online_ds = MyDataset(online_test, [0, 0, 0, 2, 0, 0, 0])
trainer.predict(online_ds)

***** Running Prediction *****
  Num examples = 7
  Batch size = 16


PredictionOutput(predictions=array([[ 2.6527088 , -3.9112282 ,  0.28650457],
       [ 2.181105  , -3.6231873 ,  0.5682687 ],
       [ 0.96109605, -1.7998203 ,  0.33854768],
       [-0.6710125 ,  2.2690775 , -1.4880042 ],
       [ 0.840981  , -3.963299  ,  3.5903444 ],
       [ 3.4300041 , -4.3258862 , -0.12243924],
       [-0.94187367,  3.4066384 , -2.2870247 ]], dtype=float32), label_ids=array([0, 0, 0, 2, 0, 0, 0], dtype=int64), metrics={'test_loss': 1.6829805374145508, 'test_runtime': 0.186, 'test_samples_per_second': 37.626, 'test_steps_per_second': 5.375})